In [2]:
# on server: 'screen' ,then start script
# use 'strg+a d' to return to terminal
# use 'screen -r' to return to screen

import numpy as np
import json
import os

from glove import Corpus, Glove

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.layers import Input,Dense,LSTM,GRU
from keras.layers import Bidirectional
from keras.utils import plot_model
# os.environ['CUDA_VISIBLE_DEVICES']='0'

import rnn_functions as rf

Using TensorFlow backend.


In [48]:
path='models'
# rnn parameters
hidden_size = 100 #100 is the standard
batch_size = 512 #for the training on the GPU this to be has to very large, otherwise the GPU is used very inefficiently
epochs = 100

#glove embedding parameters
glove_dir = '../glove/glove.6B.100d.txt'
embedding_dim = 100

In [52]:
#open SQuAD-dataset and extract the relevant data from the json-file
#to a easier readable/accessible dictionary
with open('SQuAD/train-v2.0.json') as file:
    train=json.load(file)
train_context=[]
train_question=[]
train_answer=[]
train_new={'context':train_context,'question':train_question,'answer':train_answer}
for j,data in enumerate(train['data']):
    for i,paragraph in enumerate(data['paragraphs']):
        context=paragraph['context']
        for qas in paragraph['qas']:
            #create a dataset with only the answerable questions
            #add a bos and eos token to the target
            if (qas['is_impossible']==False):
                a=context.lower()
                b=qas['question'].lower()
                c=qas['answers'][0]['text'].lower()
                
                train_new['context'].append(a)
                train_new['question'].append(b)
                train_new['answer'].append('\t'+c+'\n')
# train_new['decoder_input']=train_new['answer']
# train_new['decoder_input'][0]='\n'
print(len(train_new['context']))
print(len(train_new['question']))
print(len(train_new['answer']))
# print(len(train_new['decoder_input']))

86821
86821
86821


In [5]:
# lines=['Hello this is a tutorial on how to convert the word in an integer format', 
#        'this is a beautiful day',
#        'Jack is going to office']
# lines = remove_stopwords(lines)
# lines = lemmatize(lines)
# lines = split_lines(lines)
# lines

In [53]:
context=rf.split_lines(rf.lemmatize(rf.remove_stopwords(train_new['context'])))
question=rf.split_lines(rf.lemmatize(rf.remove_stopwords(train_new['question'])))
answer=rf.split_lines(rf.lemmatize(rf.remove_stopwords(train_new['answer'])))

100%|██████████| 86821/86821 [00:00<00:00, 94068.14it/s] 


In [8]:
max_len_context=max(map(len,context))
max_len_question=max(map(len,question))
max_len_answer=max(map(len,answer))
print(max_len_context,max_len_question,max_len_answer)

416 31 23


In [16]:
# train_all=[]
# for line in train_new['context']:
#     train_all.append(line)
# for line in train_new['question']:
#     train_all.append(line)
# for line in train_new['answer']:
#     train_all.append(line)

In [36]:
# vocab = set()
# for text in train_all:
#     vocab |= set(text_to_word_sequence(text))
# vocab = sorted(vocab)
# vocab_size = len(vocab)+1
# print(np.shape(vocab))

(91925,)


In [54]:
# #prepare the data to use as input of the rnn
# tokenizer = Tokenizer(num_words=vocab_size)
# tokenizer.fit_on_texts(train_all)
# context_sequences = tokenizer.texts_to_sequences(train_new['context'])
# question_sequences = tokenizer.texts_to_sequences(train_new['question'])
# answer_sequences = tokenizer.texts_to_sequences(train_new['answer'])

# max_len_context=max(map(len,context_sequences))
# max_len_question=max(map(len,question_sequences))
# max_len_answer=max(map(len,answer_sequences))

# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))

# context = pad_sequences(context_sequences, maxlen=max_len_context)
# question = pad_sequences(question_sequences, maxlen=max_len_question)
# answer = pad_sequences(answer_sequences, maxlen=max_len_answer)

# print('Shape of context tensor:', context.shape)
# print('Shape of question tensor:', question.shape)
# print('Shape of answer tensor:', answer.shape)

In [38]:
# # creating a corpus object
# corpus = Corpus() 
# #training the corpus to generate the co occurence matrix which is used in GloVe
# corpus.fit(question, window=10)
# #creating a Glove object which will use the matrix created in the above lines to create embeddings
# #We can set the learning rate as it uses Gradient Descent and number of components
# glove = Glove(no_components=25, learning_rate=0.05)
 
# glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
# glove.add_dictionary(corpus.dictionary)
# glove.save('glove_models/question_glove.model')

# # creating a corpus object
# corpus = Corpus() 
# #training the corpus to generate the co occurence matrix which is used in GloVe
# corpus.fit(answer, window=10)
# #creating a Glove object which will use the matrix created in the above lines to create embeddings
# #We can set the learning rate as it uses Gradient Descent and number of components
# glove = Glove(no_components=25, learning_rate=0.05)
 
# glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
# glove.add_dictionary(corpus.dictionary)
# glove.save('glove_models/answer_glove.model')

# # creating a corpus object
# corpus = Corpus() 
# #training the corpus to generate the co occurence matrix which is used in GloVe
# corpus.fit(context, window=10)
# #creating a Glove object which will use the matrix created in the above lines to create embeddings
# #We can set the learning rate as it uses Gradient Descent and number of components
# glove = Glove(no_components=25, learning_rate=0.05)

# glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
# glove.add_dictionary(corpus.dictionary)
# glove.save('glove_models/context_glove.model')

In [39]:
# context_glove=Glove().load('glove_models/context_glove.model')
# question_glove=Glove().load('glove_models/question_glove.model')
# answer_glove=Glove().load('glove_models/answer_glove.model')

In [40]:
# context_embedded=rf.fit_vectors(context,context_glove)
# question_embedded=rf.fit_vectors(question,question_glove)
# answer_embedded=rf.fit_vectors(answer,answer_glove)
# print(np.shape(context_embedded))
# print(np.shape(question_embedded))
# print(np.shape(answer_embedded))

In [41]:
# #FIX_ME: add glove download
# # https://nlp.stanford.edu/projects/glove/
# #get glove embeddings
# embeddings_index = {}
# f = open(glove_dir)
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [42]:
# #extract the glove-embedding to a matrix
# embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
# np.shape(embedding_matrix)

(91926, 100)

In [44]:
max_len_decoder_inputs=200

In [55]:
# # Define an input sequence and process it.
# context_encoder_inputs = Input(shape=(None,))
# context_x = Embedding(vocab_size, embedding_dim,weights=[embedding_matrix],
#                       trainable=False)(context_encoder_inputs)
# context_x, context_state_h, context_state_c = LSTM(embedding_dim,
#                            return_state=True)(context_x)
# context_encoder_states = [context_state_h, context_state_c]

# question_encoder_inputs = Input(shape=(None,))
# question_x = Embedding(vocab_size, embedding_dim,weights=[embedding_matrix],
#                        trainable=False)(question_encoder_inputs)
# question_x, question_state_h, question_state_c = LSTM(embedding_dim,
#                            return_state=True)(question_x)
# question_encoder_states = [question_state_h, question_state_c]

# state_h=layers.Concatenate()([context_state_h,question_state_h])
# state_c=layers.Concatenate()([context_state_c,question_state_c])
# concat_states=[state_h,state_c]

# # Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(None,))
# x = Embedding(vocab_size, embedding_dim,weights=[embedding_matrix],trainable=False)(decoder_inputs)
# x = LSTM(embedding_dim*2, return_sequences=True)(x, initial_state=concat_states)
# decoder_outputs = Dense(max_len_decoder_inputs, activation='softmax')(x)

# # Define the model that will turn
# # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = Model([context_encoder_inputs,question_encoder_inputs, decoder_inputs], decoder_outputs)

# # Compile & run training
# model.compile(optimizer='adam', loss='categorical_crossentropy')
# model.summary()

In [56]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [50]:
# plot_model(model, to_file=path+'/model.png')

In [ ]:
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!

# model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)

In [ ]:
# print('Training')
# #flatten y_train
# model.fit([x_train_context, x_train_question], y_train_answer,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.05)

# print('Evaluation')
# loss, acc = model.evaluate([x_eval_context, x_eval_question], y_eval_answer,
#                            batch_size=batch_size)
# print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

In [ ]:
# print('save model')
# # plot_model(model, to_file=path+'/model.png')
# model.save_weights('models/baseline/baseline_model.h5') #save weights
# model_json = model.to_json()
# with open("models/baseline/baseline_model.json",'w') as json_file:
#     json_file.write(model_json)